# Постановка задачи

В рамках задачи <a href="https://city-mobil.atlassian.net/browse/DI-917">DI-917</a> возникла необходимость проверить изменение конверсии водителей из регистрации в первую поездку в связи с проведенными там экспериментами с выколотыми лесенками. Поскольку нет способа, позволяющего однозначно ответить на вопрос "как повлияли эксперименты на привлечение водителей", проверил изменение метрики данной конверсии до начала экспериментов и после. При отсутствии негативных изменений это позволит с некоторой вероятностью утверждать, что негативного влияния эксперименты не оказали

## Получение данных

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import logging
import sys
import json
import datetime
import itertools
import datetime
import plotly.express as px

import pyexasol
from citymobil_python_mysql_wrapper import MysqlWrapper
from citymobil_python_clickhouse_wrapper import ClickHouseWrapper

import random
from scipy.stats import norm, ttest_ind, mannwhitneyu, t
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.multitest import multipletests

In [ ]:
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [3]:
cred = pd.read_json(r'/Users/skostuchik/crd_exa.json')
user = cred.iloc[0, 0]
password = cred.iloc[0, 1]
mysql_password = cred.iloc[0, 2]
ch_password = cred.password_ch.squeeze()

In [4]:
#Exasol
C = pyexasol.connect(dsn='ex1..3.city-srv.ru:8563', user=user, password=password, fetch_dict=True)

In [8]:
data_sql = '''
    SELECT
      to_date(dcf.first_driver_created_dttm) AS dt
         , to_date(date_trunc('week', dcf.first_driver_created_dttm)) week
           , case when local.dt < '2021-10-11' then 'before' else 'after' end period
      , count(distinct case when dcf.FIRST_DRIVER_CREATED_DTTM is not null then dcf.FIRST_DRIVER_RK end) AS "Регистраций"
      , count(distinct case when dcf.FIRST_ACTIVATION_DTTM is not null then dcf.FIRST_DRIVER_RK end )    AS "Активировано"
      , count(distinct case when dcf.FIRST_AUTOACCEPT_DTTM is not null then dcf.FIRST_DRIVER_RK end)     AS "Выход на линию"
      , count(distinct case when dcf.first_suggest_dttm is not null then dcf.FIRST_DRIVER_RK end)        AS "Предложили заказ"
      , count(distinct case when dcf.first_suggest_accept_dttm is not null then dcf.FIRST_DRIVER_RK end) AS "Принятие заказа"
      , count(distinct case when dcf.first_assign_driver_dttm is not null then dcf.FIRST_DRIVER_RK end)  AS "Назначение на заказ"
      , count(distinct case when dcf.first_rc_status_dttm is not null then dcf.FIRST_DRIVER_RK  end)     AS "Водитель прибыл"
      , count(distinct case when dcf.first_ow_status_dttm is not null then dcf.FIRST_DRIVER_RK  end)     AS "Водитель в пути"
      , count(distinct case when dct.TRIP_NUM = 1 then FIRST_DRIVER_RK  end)                             AS "FTs"
      , count(distinct case when dct.TRIP_NUM = 50 then FIRST_DRIVER_RK  end)                            AS "50T"
      , count(distinct case when dct.TRIP_NUM = 100 then FIRST_DRIVER_RK end)                            AS "100T"
    FROM REPLICA_MART.driver_chain_funnel dcf
      LEFT JOIN EMART.DRIVER_CHAIN_TRIP dct
        ON dct.chain_num = dcf.chain_num
        AND dct.phone_num = dcf.phone_num
    WHERE dcf.first_driver_created_dttm >= DATE_TRUNC('month', add_months('2021-10-11', - 2))
        and dcf.first_driver_created_dttm < current_date
    GROUP BY 1, 2, 3
    order BY 1
'''

In [9]:
data = pd.DataFrame(C.execute(data_sql).fetchall())

In [76]:
data['daily_cr'] = data['FTs'] / data['Регистраций']

In [63]:
weekly = data.groupby(['PERIOD', 'WEEK'])['Регистраций','FTs'].sum().reset_index()

<ipython-input-63-d5574773a27f>:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [64]:
weekly['week_in_period'] = weekly.sort_values(by='WEEK', ascending="True").groupby('PERIOD').cumcount() +1
weekly['CR'] = weekly['FTs'] / weekly['Регистраций']
weekly['WEEK'] = weekly['WEEK'].apply(lambda x: pd.Timestamp(x).date())

In [65]:
weekly = weekly.sort_values(by='WEEK', ascending="True")

## Недельная динамика конверсии до и после эксперимента

Периоды проведения экспериментов с соответствующими id акций выделены цветом на графиках

In [70]:
fig = px.line(weekly, x="WEEK", y="CR",
              title='Weekly Dynamic')
fig.add_hline(y=weekly[weekly['PERIOD']=='before']['FTs'].sum() / weekly[weekly[
    'PERIOD']=='before']['Регистраций'].sum(), line_dash="dot",
              annotation_text="Baseline Before Experements", 
              annotation_position="top left",
              annotation_font_size=20,
              annotation_font_color="black")
fig.add_vrect(x0="2021-10-11", x1="2021-10-20", 
              annotation_text="7215 & 7211", annotation_position="top left",
              annotation=dict(font_size=20, font_family="Helvetica"),
              fillcolor="green", opacity=0.25, line_width=0)
fig.add_vrect(x0="2021-10-21", x1="2021-11-02", 
              annotation_text="7207", annotation_position="bottom left",
              annotation=dict(font_size=20, font_family="Helvetica"),
              fillcolor="blue", opacity=0.25, line_width=0)
fig.add_vrect(x0="2021-10-21", x1="2021-12-20", 
              annotation_text="7411", annotation_position="top right",
              annotation=dict(font_size=20, font_family="Helvetica"),
              fillcolor="yellow", opacity=0.25, line_width=0)
fig.show()

Сравним период к периоду до и после, где каждой неделе был присвоен соответствующий порядковый номер внутри периода

In [74]:
fig = px.line(weekly, x="week_in_period", y="CR", color='PERIOD',
              title='Weekly Dynamic Period Comparing')

fig.add_hline(y=weekly[weekly['PERIOD']=='before']['FTs'].sum() / weekly[weekly[
    'PERIOD']=='before']['Регистраций'].sum(), line_dash="dot",
              annotation_text="Average Before Experements", 
              annotation_position="bottom left",
              annotation_font_size=20,
              annotation_font_color="black")
fig.add_hline(y=weekly[weekly['PERIOD']=='after']['FTs'].sum() / weekly[weekly[
    'PERIOD']=='after']['Регистраций'].sum(), line_dash="dot",
              annotation_text="Average After Experements", 
              annotation_position="top right",
              annotation_font_size=20,
              annotation_font_color="black")

fig.show()

## Дневная динамика конверсии до и после эксперимента

Периоды проведения экспериментов с соответствующими id акций выделены цветом на графиках

In [80]:
fig = px.line(data, x="DT", y="daily_cr",
              title='Daily Dynamic')

fig.add_hline(y=data[data['PERIOD']=='before']['FTs'].sum() / data[data[
    'PERIOD']=='before']['Регистраций'].sum(), line_dash="dot",
              annotation_text="Baseline Before Experements", 
              annotation_position="top left",
              annotation_font_size=20,
              annotation_font_color="black")
fig.add_vrect(x0="2021-10-11", x1="2021-10-20", 
              annotation_text="7215 & 7211", annotation_position="top left",
              annotation=dict(font_size=20, font_family="Helvetica"),
              fillcolor="green", opacity=0.25, line_width=0)
fig.add_vrect(x0="2021-10-21", x1="2021-11-02", 
              annotation_text="7207", annotation_position="bottom left",
              annotation=dict(font_size=20, font_family="Helvetica"),
              fillcolor="blue", opacity=0.25, line_width=0)
fig.add_vrect(x0="2021-10-21", x1="2021-12-20", 
              annotation_text="7411", annotation_position="top right",
              annotation=dict(font_size=20, font_family="Helvetica"),
              fillcolor="yellow", opacity=0.25, line_width=0)

fig.show()

# Итого:

Конверсия из регистрации в первую поездку за период с начала экспериментов не стала ниже. Более того, она существенно увеличилась. На привлечение водителей влияет множество других факторов, помимо экспериментов. Однако говрить о том, что эксперименты навредили CR регистрация/поездка поводов нет